In [ ]:
!pip install tensorflow
!pip install tensorflowjs

In [ ]:
#Importación de las librerías necesarias para el laboratorio
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.utils import to_categorical

AttributeError: module 'numpy' has no attribute '_no_nep50_warning'

In [ ]:
#Imprimir las propiedades de Tensorflow en la VM
!pip show tensorflow

#Se obtienen los datos de entrenamiento y prueba en dos arreglos que contienen la información de las imágenes y las etiquetas
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Inicializar el plot
plt.figure(figsize=(15, 3))

# Cada Item tiene la configuración: 1 fila, 5 columnas y los items inician en 1 al inicio del ciclo for
plotItem = 151

for item in range(5):
    # Posición del item en el plot
    plt.subplot(plotItem)
    # Se obtiene el máximo de elementos de x_train
    max_value = x_train.shape[0]
    # Aleatoriamente se obtiene un item de x_train
    random_item = random.randint(0, max_value)
    # Configuración para la impresión de la figura en escala de grises
    plt.imshow(x_train[random_item], cmap=plt.get_cmap('gray'))
    # Se añade la etiqueta que identifica a cada item como título
    plt.title(f"Etiquetado: {y_train[random_item]}")
    # Incrementamos la configuración de la figura en el plot
    plotItem += 1
    
plt.show()

print('Existen {} elementos dentro del arreglo x_train.'.format(x_train.shape[0]))
print('La altura de cada imagen es de {} píxeles.'.format(x_train.shape[1]))
print('El ancho de cada imagen es de {} píxeles.'.format(x_train.shape[2]))


# Calcula el número total de píxeles en cada imagen, se obtiene el número total de píxeles por imagen: 784
num_pixels = x_train.shape[1] * x_train.shape[2]
# Cambiar la forma de x_train y x_test a una forma de 4 dimensiones con un tipo float32
x_train = x_train.reshape((x_train.shape[0], 28, 28, 1)).astype('float32')
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1)).astype('float32')

# Normalizar las entradas de 0-255 (imagen RGB) a 0-1
x_train = x_train / 255
x_test = x_test / 255

# Codificación one-hot, cada etiqueta se representa como un vector con un único valor 1 en la posición correspondiente a la clase y 0s en todas las demás posiciones.
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Obtener el número de clases (dimensionalidad de la codificación one-hot):
num_classes = y_test.shape[1]

# Crea una instancia de la clase Sequential de Keras. Un modelo secuencial es un tipo de modelo en el que las capas se apilan secuencialmente, es decir, una capa sigue a la anterior en un orden lineal.
model = Sequential()

# Añade una capa de convolución 2D al modelo secuencial. Esta capa aplicará 30 filtros de tamaño 5x5 a las imágenes de entrada de tamaño 28x28 píxeles con 1 canal. La salida de esta capa se pasará a través de la función de activación ReLU para introducir no linealidades.
model.add(Conv2D(30, (5, 5), input_shape=(28, 28, 1), activation='relu'))
# MaxPooling2D ayuda a reducir la resolución de las características espaciales y a mejorar la eficiencia y robustez del modelo. Max pooling es una técnica para reducir las dimensiones espaciales de las características, manteniendo la información más relevante.
model.add(MaxPooling2D())
# Añade una capa de convolución con 15 filtros de tamaño 3x3, utilizando la función de activación ReLU para extraer características adicionales de las imágenes en el modelo.
model.add(Conv2D(15, (3, 3), activation='relu'))
model.add(MaxPooling2D())
# Añade una capa de regularización al modelo, apagando el 20% de las neuronas durante el entrenamiento para ayudar a prevenir el sobreajuste y mejorar la capacidad del modelo para generalizar a nuevos datos.
model.add(Dropout(0.2))
# Transforma la salida multidimensional de las capas anteriores en un vector unidimensional, lo que permite que esta salida sea utilizada por capas densas para la clasificación o regresión final. Es un paso esencial en la transición de las características extraídas a la fase de decisión en redes neuronales convolucionales.
model.add(Flatten())
# Añade una capa completamente conectada con 128 y 50 neuronas al modelo, donde cada neurona usa la función de activación ReLU. Esta capa es clave para aprender representaciones complejas de los datos y para realizar la clasificación o regresión final en el modelo.
model.add(Dense(128, activation='relu'))
model.add(Dense(50, activation='relu'))
# Añade una capa de salida al modelo que tiene num_classes neuronas, cada una de las cuales representa una clase en el problema de clasificación. La función de activación softmax convierte los valores de salida en probabilidades que suman 1, facilitando la clasificación en múltiples categorías.
model.add(Dense(num_classes, activation='softmax'))
# Configura el modelo para el entrenamiento especificando cómo calcular la pérdida (categorical_crossentropy para clasificación multiclase), qué algoritmo de optimización usar (adam), y qué métrica usar para evaluar el rendimiento del modelo (precisión).
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#batch_size: Determina cuántas muestras se procesan en una sola iteración antes de actualizar los pesos del modelo. Afecta la velocidad y la estabilidad del entrenamiento.

#verbose: Controla el nivel de detalle que se muestra durante el entrenamiento. verbose=2 muestra información detallada al final de cada época, proporcionando una visión clara del progreso del entrenamiento.

In [ ]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=1, batch_size=200, verbose=2)

# Mide el rendimiento del modelo en el conjunto de datos de prueba, calculando la pérdida y las métricas definidas durante la compilación del modelo. La variable scores almacena estos resultados, permitiendo evaluar la calidad del modelo después del entrenamiento.
scores = model.evaluate(x_test, y_test, verbose=0)

# Calcula y muestra la tasa de error del modelo basado en su precisión. El error de referencia es la tasa de error del modelo, que se obtiene al restar la precisión en porcentaje de 100%. Esta medida permite comparar el rendimiento del modelo con una solución básica y evaluar su efectividad.
print('Baseline Error: {}'.format(100-scores[1]*100))

In [ ]:
tf.saved_model.save(model, "./model")

In [ ]:
!tensorflowjs_converter --input_format=keras --output_format=tfjs_layers_model -quantize_float16 --weight_shard_size_bytes=4194304 ./model/model.h5 ./modeljs